Test and execution code to produce Gauss Legendre Quadrature roots and weights per degree n. This is quite messy, so definitely feel free to view the pdf for an in-depth analysis.

In [348]:
import Pkg; 
Pkg.add("Roots")
Pkg.add("DataFrames")
Pkg.add("IJulia")
Pkg.add("PrettyTables")
Pkg.add("QuadGK")
Pkg.add("FastGaussQuadrature")
Pkg.add("LegendrePolynomials")
Pkg.add("QuadGK")
Pkg.add("SymPy")

   Resolving package versions...
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\CT\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\CT\.julia\environments\

In [2]:
using Roots, PrettyTables, DataFrames, IJulia

In [3]:
using Test, FastGaussQuadrature, LegendrePolynomials, LinearAlgebra, QuadGK, SymPy

In [5]:
# find legendre poly for degree n
function legendre(n, x)
    if n <= -1 || typeof(n) != Int64 throw(DomainError(n, "n must be non-negative integer!")) end
    if n == 0 return 1.0  # base case: p0(x) = 1
    elseif n == 1 return x  # base case: p1(x) = x
    else
        # three-term recurrence formula to calculate Legendre polynomial pn(x) for n
        return ((2 * n - 1) * x * legendre(n - 1, x) - (n - 1) * legendre(n - 2, x)) / n
    end
end

# calculate derivative of Legendre polynomial for degree n
function legendre_derivative(n, x)
    if n <= -1 || typeof(n) != Int64 throw(DomainError(n, "n must be non-negative integer!")) end
    if n == 0 return 0.0  # derivative of 1 is 0
    else
        # recursive formula to calculate derivative
        return n * (x * legendre(n, x) - legendre(n - 1, x)) / (x^2 - 1)
    end
end

# find Legendre polynomial roots and weights
function legendre_roots_and_weights(n)   
    if n <= -1 || typeof(n) != Int64 throw(DomainError(n, "n must be non-negative integer!")) end
    # Use the roots library to find the roots of Legendre polynomial pn(x)
    roots = find_zeros(x -> legendre(n,x), -1.0, 1.0)
    
    # calculate the weights of the Legendre polynomial roots using weights formula
    weights = [2 / ((1 - root^2) * (legendre_derivative(n, root))^2) for root in roots]
    
    return roots, weights
end

legendre_roots_and_weights (generic function with 1 method)

In [6]:
r,w = legendre_roots_and_weights(5)

([-0.906179845938664, -0.538469310105683, 0.0, 0.538469310105683, 0.906179845938664], [0.236926885056189, 0.47862867049936636, 0.5688888888888889, 0.47862867049936636, 0.236926885056189])

In [7]:
df = DataFrame(Degree = String[], Roots = Float64[], Weights = Float64[]) # store data
n = 10                                                                    # set max degree
last_degree = 0                                                           # track degree n

# calculate and add the roots and weights for degrees 2 to 10
for n in 2:n
    roots, weights = legendre_roots_and_weights(n)
    
    # add the roots and weights for the current degree
    for (root, weight) in zip(roots, weights)                            # pair roots and weights
        if n != last_degree
            push!(df, [string(n), root, weight])                         # add data with degree n
            last_degree = n                                              # switch degree if needed
        else
            push!(df, ["", root, weight])                                # makes degree col cleaner
        end
    end
end

#show(df, show_row_number = false, allcols=true, allrows=true)
pretty_table(df)                                                        # convert to a pretty table :D

┌────────┬───────────┬───────────┐
│ Degree │     Roots │   Weights │
│ String │   Float64 │   Float64 │
├────────┼───────────┼───────────┤
│      2 │  -0.57735 │       1.0 │
│        │   0.57735 │       1.0 │
│      3 │ -0.774597 │  0.555556 │
│        │       0.0 │  0.888889 │
│        │  0.774597 │  0.555556 │
│      4 │ -0.861136 │  0.347855 │
│        │ -0.339981 │  0.652145 │
│        │  0.339981 │  0.652145 │
│        │  0.861136 │  0.347855 │
│      5 │  -0.90618 │  0.236927 │
│        │ -0.538469 │  0.478629 │
│        │       0.0 │  0.568889 │
│        │  0.538469 │  0.478629 │
│        │   0.90618 │  0.236927 │
│      6 │  -0.93247 │  0.171324 │
│        │ -0.661209 │  0.360762 │
│        │ -0.238619 │  0.467914 │
│        │  0.238619 │  0.467914 │
│        │  0.661209 │  0.360762 │
│        │   0.93247 │  0.171324 │
│      7 │ -0.949108 │  0.129485 │
│        │ -0.741531 │  0.279705 │
│        │ -0.405845 │   0.38183 │
│        │       0.0 │  0.417959 │
│        │  0.405845

Test functions to make sure they have correct outputs

In [8]:
# example 1 from B&F 4.7
r,w = legendre_roots_and_weights(3)    # find roots and weights
h(x) = exp(x)*cos(x)                        # define h(x)
I = dot(w, h.(r))                           # evaluate integral

1.9333904692642967

In [11]:
#f(x) = 1/(2+x)
#f(x) = 0.5*exp(-((1+x)/2)^2)
f(x) = exp(x)*cos(x)
I = quadgk(f, -1, 1)[1]                           # compute integral using package

println("Expected solution: $I\n")                # solution to integral

println("Gauss-Legendre Approximation:")
for n = 2:10
    r, w = legendre_roots_and_weights(n)
    println("n = $n: I = ", dot(w, f.(r)))        # approx per degree n
end

Expected solution: 1.9334214962007135

Gauss-Legendre Approximation:
n = 2: I = 1.9629727607543537
n = 3: I = 1.9333904692642967
n = 4: I = 1.9334168941640448
n = 5: I = 1.933421497268504
n = 6: I = 1.9334214962992713
n = 7: I = 1.9334214962007055
n = 8: I = 1.9334214962007126
n = 9: I = 1.9334214962007132
n = 10: I = 1.9334214962007141


In [306]:
my_roots_weights = []                                             # lists to store data
expected_roots_weights = []                                       

# not clean and not optimized, but works enough
for n = 2:10
    my_roots, my_weights = legendre_roots_and_weights(n)          # call my function
    exp_roots, exp_weights = gausslegendre(n)                     # function from package
    
    for (my_root, my_weight) in zip(my_roots, my_weights)         # whatever i get
        push!(my_roots_weights, [my_root, my_weight])
    end
    
    for (exp_root, exp_weight) in zip(exp_roots, exp_weights)     # what's expected
        push!(expected_roots_weights, [exp_root, exp_weight])
    end
end

"UNIT TESTING"

In [310]:
@testset "Gauss-Legendre" begin
    
    @test_throws DomainError legendre_roots_and_weights(-2)        # check error cases
    @test_throws DomainError legendre_roots_and_weights(5.5)  
    
    @test legendre(6, 0.5) ≈ Pl(0.5, 6)                            # legendre poly n=6
    @test legendre_derivative(6, 0.5) ≈ dnPl(0.5, 6, 1)            # first derivative

    exp_roots, exp_weights = gausslegendre(6)                      # external package
    my_roots, my_weights = legendre_roots_and_weights(6)           # my function

    @test exp_roots ≈ my_roots
    @test exp_weights ≈ my_weights
    
    g(x) = x^4-4x^2+12                                             # another integral
    I = dot(my_weights, g.(my_roots))
    @test I ≈ 326/15
    
    @test my_roots_weights ≈ expected_roots_weights                # check from n=2-10
    
end

Test Summary:  | Pass  Total  Time
Gauss-Legendre |    8      8  0.1s


Test.DefaultTestSet("Gauss-Legendre", Any[], 8, false, false, true, 1.69931028071e9, 1.699310280807e9, false)

Quite slow compared to an already tested library/package

In [269]:
@time roots, weights = gausslegendre(10)
@time _roots, _weights = legendre_roots_and_weights(10)

  0.000028 seconds (13 allocations: 1.297 KiB)
  0.002301 seconds (2.83 k allocations: 71.578 KiB)


([-0.9739065285171716, -0.8650633666889844, -0.6794095682990243, -0.43339539412924716, -0.1488743389816312, 0.1488743389816312, 0.43339539412924716, 0.6794095682990243, 0.8650633666889844, 0.9739065285171716], [0.06667134430868844, 0.14945134915058064, 0.2190863625159821, 0.26926671930999624, 0.2955242247147529, 0.2955242247147529, 0.26926671930999624, 0.2190863625159821, 0.14945134915058064, 0.06667134430868844])

In [259]:
@time roots, weights = gausslegendre(25)
@time _roots, _weights = legendre_roots_and_weights(25)

  0.000032 seconds (13 allocations: 2.969 KiB)
  6.111748 seconds (6.10 k allocations: 152.406 KiB)


([-0.995556969790498, -0.9766639214595174, -0.9429745712289743, -0.8949919978782753, -0.833442628760834, -0.7592592630373576, -0.6735663684734683, -0.577662930241223, -0.473002731445715, -0.3611723058093878  …  0.3611723058093878, 0.473002731445715, 0.577662930241223, 0.6735663684734683, 0.7592592630373576, 0.833442628760834, 0.8949919978782753, 0.9429745712289743, 0.9766639214595174, 0.995556969790498], [0.011393798501026533, 0.026354986615032334, 0.04093915670130625, 0.054904695975835326, 0.06803833381235701, 0.08014070033500101, 0.09102826198296372, 0.1005359490670506, 0.10851962447426362, 0.11485825914571159  …  0.11485825914571159, 0.10851962447426362, 0.1005359490670506, 0.09102826198296372, 0.08014070033500101, 0.06803833381235701, 0.054904695975835326, 0.04093915670130625, 0.026354986615032334, 0.011393798501026533])

FASTERRRR... can use dynamic programming techniques to produce a faster execution time

In [311]:
function faster_legendre(n, x)
    if n == 0
        return 1.0
    elseif n == 1
        return x
    else
        p0, p1 = 1.0, x
        for k in 2:n
            p0, p1 = p1, ((2 * k - 1) * x * p1 - (k - 1) * p0) / k
        end
        return p1
    end
end

function faster_legendre_derivative(n, x)
    if n == 0
        return 0.0
    elseif n == 1
        return 0.0
    else
        p0, p1 = 1.0, x
        dp0, dp1 = 0.0, 0.0
        for k in 2:n
            p0, p1 = p1, ((2 * k - 1) * x * p1 - (k - 1) * p0) / k
            dp0, dp1 = dp1, k * (x * p1 - p0) / (x^2 - 1)
        end
        return dp1
    end
end

function faster_legendre_roots_and_weights(n)
    if n < 0 || typeof(n) != Int64
        throw(DomainError("n must be a non-negative integer."))
    end

    # Use the roots library to find the roots of Legendre polynomial pn(x)
    roots = find_zeros(x -> faster_legendre(n, x), -1.0, 1.0)

    # Calculate the weights of the Legendre polynomial roots using weights formula
    weights = [2 / ((1 - root^2) * faster_legendre_derivative(n, root)^2) for root in roots]

    return roots, weights
end


faster_legendre_roots_and_weights (generic function with 1 method)

In [312]:
faster_legendre_roots_and_weights(3)

([-0.7745966692414834, 0.0, 0.7745966692414834], [0.5555555555555551, 0.8888888888888888, 0.5555555555555551])

In [313]:
@testset "Gauss-Legendre" begin
    
    @test_throws DomainError faster_legendre_roots_and_weights(-2)    # check error cases
    @test_throws DomainError faster_legendre_roots_and_weights(5.5)  
    
    @test faster_legendre(6, 0.5) ≈ Pl(0.5, 6)                        # legendre poly n=6
    @test faster_legendre_derivative(6, 0.5) ≈ dnPl(0.5, 6, 1)        # first derivative

    exp_roots, exp_weights = gausslegendre(6)                       # external package
    my_roots, my_weights = faster_legendre_roots_and_weights(6)       # my function

    @test exp_roots ≈ my_roots
    @test exp_weights ≈ my_weights
    
    f(x) = x^2                                                     # check integral
    I = dot(my_weights, f.(my_roots))
    @test I ≈ 2/3
    
    g(x) = x^4-4x^2+12                                             # another integral
    I = dot(my_weights, g.(my_roots))
    @test I ≈ 326/15
    
end

Test Summary:  | Pass  Total  Time
Gauss-Legendre |    8      8  0.1s


Test.DefaultTestSet("Gauss-Legendre", Any[], 8, false, false, true, 1.699332269223e9, 1.699332269334e9, false)

In [297]:
roots, weights = gausslegendre(25)
_roots, _weights = faster_legendre_roots_and_weights(25)

@test roots ≈ _roots
@test weights ≈ _weights

Test Passed

WAYYY faster than previous code, but still not optimally efficient...

In [321]:
@time roots, weights = gausslegendre(5000)
@time _roots, _weights = faster_legendre_roots_and_weights(5000)

  0.000092 seconds (10 allocations: 147.375 KiB)
  6.507521 seconds (155.50 k allocations: 14.313 MiB)


([-0.9999955422488499, -0.9998886778576975, -0.9997334698008677, -0.999622273543701, -0.9995310108404106, -0.9990942709215108, -0.9989556757718179, -0.9988677870779505, -0.998776350060068, -0.9986813650429861  …  0.9986813650429861, 0.998776350060068, 0.9988677870779505, 0.9989556757718179, 0.9990942709215108, 0.9995310108404106, 0.999622273543701, 0.9997334698008677, 0.9998886778576975, 0.9999955422488499], [1.8748524164843578e-6, 9.373901393841853e-6, 1.4504128471175523e-5, 1.7266179515159774e-5, 1.923887178290447e-5, 2.6733205377401842e-5, 2.8704808912935534e-5, 2.9887636320170282e-5, 3.1070357556196205e-5, 3.225296841957945e-5  …  3.225296841957945e-5, 3.1070357556196205e-5, 2.9887636320170282e-5, 2.8704808912935534e-5, 2.6733205377401842e-5, 1.923887178290447e-5, 1.7266179515159774e-5, 1.4504128471175523e-5, 9.373901393841853e-6, 1.8748524164843578e-6])